In [2]:
import sys

sys.path.append("../modules")

import pushift_wrap as rapi
import os
import requests
import time
import random
import json
from datetime import datetime

data_path = os.path.join('D:/', 'data', 'reddit')

sub_end = "https://api.pushshift.io/reddit/search/submission/"
subcomment_end = "https://api.pushshift.io/reddit/submission/comment_ids/"
comment_end = "https://api.pushshift.io/reddit/search/comment/"

In [3]:
#load existing data
filename = 'reddit_rdenmark_q=danmark_01012020-15032021_not-finished.json'

with open(os.path.join(data_path, filename), 'r', encoding = 'utf-8') as f:
    submissions = json.load(f)

In [4]:
start_time_epoch = int(datetime(2020,1,1,0,0).timestamp())
end_time_epoch = int(datetime(2021,3,15,0,0).timestamp())

In [5]:
from json import JSONDecodeError

interval = 7 * 86400 # a week
from_time_epoch = start_time_epoch

submissions = []
while from_time_epoch <= end_time_epoch:
    
    to_time_epoch = from_time_epoch + interval 
    
    try:
        new_submissions = rapi.get_submissions(subreddit = 'denmark', before = to_time_epoch, after = from_time_epoch, q = "danmark", num_comments = 1)
        submissions = submissions + new_submissions
    
        from_time_epoch = to_time_epoch
        
        time.sleep(random.uniform(0.03, 0.10))
    except JSONDecodeError:
        time.sleep(random.uniform(0.03, 0.10))
        continue

In [4]:
len(submissions)

1848

In [5]:
sum([submission.get('num_comments') for submission in submissions])

70807

In [8]:
submissions[0].keys()

dict_keys(['author', 'created_utc', 'domain', 'full_link', 'gildings', 'id', 'is_original_content', 'is_reddit_media_domain', 'locked', 'media_only', 'num_comments', 'num_crossposts', 'over_18', 'permalink', 'pinned', 'retrieved_on', 'score', 'selftext', 'spoiler', 'stickied', 'subreddit', 'subreddit_id', 'subreddit_subscribers', 'subreddit_type', 'thumbnail', 'title', 'total_awards_received', 'url'])

In [9]:
#submissions[0]

In [6]:
from json import JSONDecodeError

for c, submission in enumerate(submissions, start = 1):
    progress = "|{0}| {1:.2f} %".format(("="*int(c/len(submissions) * 50)).ljust(50), c/len(submissions) * 100)
    if 'comments' in submission:
        print(progress, end = "\r")
        continue
    
    try:
        subid = submission.get('id')
        commentids = requests.get(f"{subcomment_end}{subid}").json().get('data')
    
        comments = rapi.get_comments(commentids)
        submission['comments'] = comments
    
        time.sleep(random.uniform(0.03, 0.10))
    except JSONDecodeError:
        print(progress, end = "\r")
        continue
        
    print(progress, end = "\r")

In [7]:
sum([len(submission.get('comments')) for submission in submissions if 'comments' in submission])

66109

In [11]:
submissions[0]['comments'][0].keys()

dict_keys(['author', 'banned_at_utc', 'body', 'collapsed', 'collapsed_reason', 'created_utc', 'edited', 'id', 'is_submitter', 'link_id', 'parent_id', 'permalink', 'retrieved_on', 'score', 'stickied', 'subreddit', 'subreddit_id', 'total_awards_received'])

In [8]:
filename = 'reddit_rdenmark_q=danmark_01012020-15032021.json'

with open(os.path.join(data_path, filename), 'w', encoding = 'utf-8') as f:
    json.dump(submissions, f)